In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import os
from llama import Workflow, Llama

os.environ["RANK"] = "0"
os.environ["WORLD_SIZE"] = "1"
os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = "29502"

workflow = Workflow.build(
    ckpt_dir='/scratch4/jeisner1/tjbai/llama_8b',
    tokenizer_path='/scratch4/jeisner1/tjbai/llama_8b/tokenizer.model',
    max_seq_len=8192,
    max_batch_size=8,
    model_parallel_size=1,
    max_nodes=100,
)

llama = Llama(workflow.model, workflow.tokenizer)

> initializing model parallel with size 1
> initializing ddp with size 1
> initializing pipeline with size 1
Loaded in 112.13 seconds


In [ ]:
from llama.workflows.prisoners import prisoners_cached, prisoners_baseline

payoff = (5, 3, 1, 0)

baseline_outputs = prisoners_baseline(llama, payoff, alice_strategy='always_defect')

In [3]:
import json

with open('/home/tbai4/llama3/dumps/prisoners/prisoners_baseline.jsonl') as f:
    data = [json.loads(line) for line in f]
    
baseline = [d['outputs'] for d in data[:100]]
always_cooperate = [d['outputs'] for d in data[100:200]]
always_defect = [d['outputs'] for d in data[200:]]

In [15]:
from llama.workflows.prisoners import baseline_nll, cached_nll

workflow.reset()

baseline_res = baseline_nll(
    llama,
    baseline[0],
    payoff=(5,3,1,0),
    alice_first=True,
    alice_strategy=None
)

cached_res = cached_nll(
    workflow,
    baseline[0],
    payoff=(5,3,1,0),
    alice_first=True,
    alice_strategy=None,
)

In [16]:
res = baseline_res
print(sum(res['alice_nll'][0]) / len(res['alice_nll'][0]))
print(sum(res['alice_nll'][1]) / len(res['alice_nll'][1]))
print(sum(res['bob_nll'][0]) / len(res['bob_nll'][0]))
print(sum(res['bob_nll'][1]) / len(res['bob_nll'][1]))

res = cached_res
print(sum(res['alice_nll'][0]) / len(res['alice_nll'][0]))
print(sum(res['alice_nll'][1]) / len(res['alice_nll'][1]))
print(sum(res['bob_nll'][0]) / len(res['bob_nll'][0]))
print(sum(res['bob_nll'][1]) / len(res['bob_nll'][1]))

1.445046552470573
1.387665540824082
1.0912827434387258
1.08309541044076
1.337704761211853
1.427836259337768
1.2268699074836724
1.1250865536961028
